In [2]:
import pandas as pd

In [29]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
df = pd.read_csv(url, names=names)

# array = df.values

In [8]:
!pip install xgboost

In [9]:
import xgboost as xgb

In [10]:
model = xgb.XGBClassifier()

614.0

In [19]:
dftrain = df[:614]
dftest = df[614:]

In [22]:
model.fit(dftrain.drop(['class'], axis=1), dftrain['class'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [23]:
model.score(dftrain.drop(['class'], axis=1), dftrain['class'])

0.8892508143322475

In [24]:
model.score(dftest.drop(['class'], axis=1), dftest['class'])

0.7532467532467533

In [30]:
# model.save_model('model_saved.digitalhouse') # from xgboost ..

In [34]:
# Exportando binario

import pickle

def save_model(model, title='model'):
    with open(title + '.pkl', 'wb') as fid:
        pickle.dump(model, fid)
        
    print('Model saved')

In [35]:
save_model(model, title='modelo_treinado')

Model saved


In [37]:
# feature engineering 

In [39]:
df.mean()

preg       3.845052
plas     120.894531
pres      69.105469
skin      20.536458
test      79.799479
mass      31.992578
pedi       0.471876
age       33.240885
class      0.348958
dtype: float64

In [49]:
temp = (df.drop(['class'],axis=1)-df.drop(['class'],axis=1).mean())/df.drop(['class'],axis=1).std()

In [52]:
temp.describe().round(4).T

,count,mean,std,min,25%,50%,75%,max
preg,768.0,0.000,1.000,-1.1411,-0.8443,-0.2508,0.6395,3.9040
plas,768.0,-0.000,1.000,-3.7812,-0.6848,-0.1218,0.6054,2.4429
pres,768.0,-0.000,1.000,-3.5703,-0.3671,0.1495,0.5629,2.7327
skin,768.0,0.000,1.000,-1.2874,-1.2874,0.1544,0.7186,4.9187
test,768.0,0.000,1.000,-0.6924,-0.6924,-0.4278,0.4117,6.6485
mass,768.0,0.000,1.000,-4.0578,-0.5952,0.0009,0.5844,4.4529
pedi,768.0,0.000,1.000,-1.1888,-0.6885,-0.2999,0.4659,5.8797
age,768.0,0.000,1.000,-1.0409,-0.7858,-0.3606,0.6598,4.0611
class,768.0,0.349,0.477,0.0000,0.0000,0.0000,1.0000,1.0000


In [51]:
temp['class'] = df['class']

In [53]:
dftrain = temp[:614]
dftest = temp[614:]

In [54]:
model.fit(dftrain.drop(['class'], axis=1), dftrain['class'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [55]:
model.score(dftrain.drop(['class'], axis=1), dftrain['class'])

0.8892508143322475

In [56]:
model.score(dftest.drop(['class'], axis=1), dftest['class'])

0.7532467532467533

In [57]:
save_model(model, title='modelo_treinado_normalizado')

Model saved


In [65]:
medias = df.drop(['class'],axis=1).mean()
desvios = df.drop(['class'],axis=1).std()

In [66]:
save_model(medias, 'medias_de_treino')
save_model(desvios, 'desvios_de_treino')

Model saved
Model saved


In [67]:
desvios

preg      3.369578
plas     31.972618
pres     19.355807
skin     15.952218
test    115.244002
mass      7.884160
pedi      0.331329
age      11.760232
dtype: float64

In [68]:
# with pipelines

In [73]:
# Create a pipeline that standardizes the data then creates a model
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier

# load data


url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(url, names=names)
array = dataframe.values


X = array[:,0:8]
Y = array[:,8]

# create pipeline

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('xgboost', XGBClassifier()))

model2 = Pipeline(estimators)

xgboost_instanciado = XGBClassifier()
# evaluate pipeline
seed = 7
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(model2, X, Y, cv=kfold)
print(results.mean())

0.7668660287081339


In [75]:
model2.fit(X, Y)

Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgboost', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, miss...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))])

In [76]:
model2.score(X, Y)

0.8723958333333334